In [16]:
import pandas as pd
import re

def clean_text_column(df, column_name):
    """Removes special characters and converts text to lowercase."""
    df[column_name] = df[column_name].str.lower()
    df[column_name] = df[column_name].apply(lambda x: re.sub(r'[^a-z0-9\s]', '', str(x)))
    return df

def standardize_phone_numbers(df, column_name):
    """Standardizes phone numbers to a common format (e.g., XXX-XXX-XXXX)."""
    df[column_name] = df[column_name].astype(str).str.replace(r'\D', '', regex=True) # Remove non-digits
    df[column_name] = df[column_name].apply(lambda x: f"{x[:3]}-{x[3:6]}-{x[6:]}" if len(x) == 10 else x)
    return df

def drop_duplicate_rows(df):
    """Removes duplicate rows based on all columns."""
    df.drop_duplicates(subset=['name', 'phone', 'comments'], inplace=True)
    return df

def anomaly_detection(data):
    """detects anomalies based on top and bottom 5 percentiles"""
    data = df['length_of_call (min)'].to_numpy()
    # Calculate the 5th percentile (lower 5%)
    lower_5_percentile = np.percentile(data, 5)

    # Calculate the 95th percentile (upper 5%, as 95% of data is below this point)
    upper_5_percentile = np.percentile(data, 95)

    print(f"The value at the 5th percentile (lower 5%) is: {lower_5_percentile}")
    print(f"The value at the 95th percentile (upper 5%) is: {upper_5_percentile}")

    # Identifying values in the lowest 5%
    lowest_5_percent_values = data[data <= lower_5_percentile]
    print(f"Values in the lowest 5% are: {lowest_5_percent_values}")

    # Identifying values in the highest 5%
    highest_5_percent_values = data[data >= upper_5_percentile]
    print(f"Values in the highest 5% are: {highest_5_percent_values}")
    

# data
df = pd.read_csv(r'C:\Users\nicks\Documents\comments.csv',encoding='latin1')

print('original:')
print()
print(df.head(10))

# Build the cleaning pipeline as a sequence of function calls
def data_cleaning_pipeline(df):
    df = drop_duplicate_rows(df)
    df = clean_text_column(df, 'comments')
    df = standardize_phone_numbers(df, 'phone')
    return df

# Apply the pipeline
print()
df.head(10)
print()
df_cleaned = data_cleaning_pipeline(df.copy()) # Use .copy() to avoid modifying original df
print('cleaned:')
print()
print(df_cleaned.head(10))
print()
print('ANOMALIES:')
print()
anomalies = anomaly_detection(df)
anomalies

original:

      name         phone           comments  length_of_call (min)
0    Alice    1234567890  Needs improvement                    23
1    Alice    1234567890  Needs improvement                     8
2      Bob  123.456.7891  Needs improvement                    12
3    Alice  123.456.7892     Great Product!                     6
4  Michael    1234567894     Great product!                     8
5  Charlie    1234567893     Great Product!                     4
6  Michael    1234567894     Great product!                     1
7     John    1234567895            Not bad                    11
8   Robert    1234567896     Great Product!                    26
9    David    1234567897            Not bad                    10


cleaned:

       name         phone           comments  length_of_call (min)
0     Alice  123-456-7890  needs improvement                    23
2       Bob  123-456-7891  needs improvement                    12
3     Alice  123-456-7892      great product      